Video 41: Logistic Regression Example Code Along
===============================================================

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mylogreg').getOrCreate()
from pyspark.ml.classification import LogisticRegression
data = spark.read.format('libsvm').load('sample_libsvm_data.txt')
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [2]:
log_reg_model = LogisticRegression()
fitted_model = log_reg_model.fit(data)
log_summary = fitted_model.summary
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111048...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126969...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170264...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200596...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695572...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102604...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215612...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [3]:
lr_train, lr_test = data.randomSplit([0.7, 0.3])
final_model = LogisticRegression()
fit_final = final_model.fit(lr_train)
prediction_and_labels = fit_final.evaluate(lr_test)
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[123,124,125...|[20.5065666501665...|[0.99999999875802...|       0.0|
|  0.0|(692,[124,125,126...|[44.8603131160058...|[1.0,3.2916492924...|       0.0|
|  0.0|(692,[126,127,128...|[26.4827780841510...|[0.99999999999684...|       0.0|
|  0.0|(692,[126,127,128...|[23.0177706175592...|[0.99999999989918...|       0.0|
|  0.0|(692,[126,127,128...|[30.0615036518216...|[0.99999999999991...|       0.0|
|  0.0|(692,[127,128,129...|[23.9248992016987...|[0.99999999995930...|       0.0|
|  0.0|(692,[127,128,129...|[22.8101042270575...|[0.99999999987592...|       0.0|
|  0.0|(692,[129,130,131...|[15.3029324844992...|[0.99999977404561...|       0.0|
|  0.0|(692,[150,151,152...|[26.3207690399766...|[0.99999999999629...|       0.0|
|  0.0|(692,[152

In [4]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
meval = BinaryClassificationEvaluator()
final_roc = meval.evaluate(prediction_and_labels.predictions)
final_roc

1.0

Video 42: Logistic Regression Code Along
===============================================================

In [5]:
data = spark.read.csv('titanic.csv', inferSchema=True, header=True)
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [ ]:
cols = data.select(['Survived',
                  'Pclass',
                  'Sex',
                  'Age',
                  'SibSp',
                  'Parch',
                  'Fare',
                  'Embarked'])
final_data = cols.na.drop()
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer
gender_indexer = StringIndexer(inputCol='Sex', outputCol='Sex_index')
gender_encoder = OneHotEncoder(inputCol='Sex_index', outputCol='Sex_vector')
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='Embark_index')
embark_encoder = OneHotEncoder(inputCol='Embark_index', outputCol='Embark_vector')
assembler = VectorAssembler(inputCols=['Pclass',
                                      'Sex_vector',
                                      'Embark_vector',
                                      'Age',
                                      'SibSp',
                                      'Parch',
                                      'Fare'],
                           outputCol='features')
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, log_reg_titanic])
train_data, test_data = final_data.randomSplit([0.7, 0.3])
train_data.describe().show()
test_data.describe().show()

In [ ]:
fit_model = pipeline.fit(train_data)
results = fit_model.transform(test_data)
results.select('Survived','prediction').show()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
meval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')
AUC = meval.evaluate(results)
AUC